In [1]:
import os
import pandas as pd
from itertools import islice
from amplpy import AMPL, Environment
from IPython.display import display

**File Paths**

In [2]:
baseexppath = '/Users/Danny/Desktop/CATEGORIES/CAREER_MANAGEMENT/CRC_ResearchScientist_Optimization/Optimization_Tool/2_ExperimentFolder/'
amplappdir = os.path.join(baseexppath, 'ampl/amplide.macosx64/')
projectpath = os.path.join(baseexppath, 'ampl/OptEfficiencySubProblem/')
datapath = os.path.join(baseexppath, 'ampl/OptEfficiencySubProblem/data/')

# Specify model and data files
# f_mod = os.path.join(baseexppath, 'ampl/example/steel3.mod')
f_mod = os.path.join(projectpath, 'test4.mod')
# f_dat = os.path.join(baseexppath, 'ampl/example/steel3.dat')
f_dat = os.path.join(projectpath, 'test4.dat')

# Specify AMPL solver to be used
f_minos_solver = os.path.join(projectpath, 'amplide.macosx64/minos')
f_gurobi_solver = os.path.join(projectpath, 'amplide.macosx64/gurobi')
f_solver = f_minos_solver

# Data table directories
sourcedatadir = os.path.join(baseexppath, 'OptSandbox/data/test_source/')
metadatadir = os.path.join(baseexppath, 'OptSandbox/data/test_metadata/')

**Set up the AMPL environment**

In [3]:
ampl = AMPL(Environment(amplappdir))
ampl.setOption('solver', f_solver)
value = ampl.getOption('solver')
print(value)

/Users/Danny/Desktop/CATEGORIES/CAREER_MANAGEMENT/CRC_ResearchScientist_Optimization/Optimization_Tool/2_ExperimentFolder/ampl/OptEfficiencySubProblem/amplide.macosx64/minos


**Read in the model**

In [4]:
ampl.read(f_mod)

**Read in the data tables**

In [5]:
# Data tables for the set definitions
TblBmp = pd.read_csv(os.path.join(sourcedatadir, 'TblBmp.csv'))
TblBmpGroup = pd.read_csv(os.path.join(sourcedatadir, 'TblBmpGroup.csv'))
TblBmpLoadSourceGroup = pd.read_csv(os.path.join(sourcedatadir, 'TblBmpLoadSourceGroup.csv'))
TblBmpType = pd.read_csv(os.path.join(sourcedatadir, 'TblBmpType.csv'))
TblLoadSource = pd.read_csv(os.path.join(sourcedatadir, 'TblLoadSource.csv'))
TblLoadSourceGroup = pd.read_csv(os.path.join(sourcedatadir, 'TblLoadSourceGroup.csv'))
TblLoadSourceGroupLoadSource = pd.read_csv(os.path.join(sourcedatadir, 'TblLoadSourceGroupLoadSource.csv'))
TblLandRiverSegment = pd.read_csv(os.path.join(sourcedatadir, 'TblLandRiverSegment.csv'))

# Data tables for the parameter definitions
TblCostBmpLand = pd.read_csv(os.path.join(metadatadir, 'TblCostBmpLand.csv'))
TblBmpEfficiency = pd.read_csv(os.path.join(sourcedatadir, 'TblBmpEfficiency.csv'))
# Target load reductions ???  (set this ourselves??)
TblLandUsePreBmp = pd.read_csv(os.path.join(sourcedatadir, 'TblLandUsePreBmp.csv'))
Tbl2010NoActionLoads = pd.read_csv(os.path.join(datapath, '2010NoActionLoads.csv'))

**SETS**

**Pollutants and Land river segments**

$P$, the set of pollutants $p=\{nitrogen, phosphorous, sediment\}$ <br>
$L$, a set of land river segments $l$

In [19]:
pltnts = ampl.getSet('PLTNTS')
pltnts.setValues(['N', 'P', 'S'])
print(pltnts.getValues())

lrsegs = ampl.getSet('LRSEGS')
lrsegids = TblLandRiverSegment[TblLandRiverSegment['landriversegment']=='N51133RL0_6450_0000'].lrsegid.tolist()
lrsegs.setValues(lrsegids)
print(lrsegs.getValues())
lrsegsetlist = list([int(x) for x in lrsegs.getValues().toPandas().index])

   PLTNTS   
    'N'        
    'P'        
    'S'        

   LRSEGS   
    1677       



**Load Sources and their groups**

$\Lambda$, the set of load sources $\lambda$ <br>
$\Psi$, the set of all load source *groups* $\psi$, where $\psi=\{\lambda_{1}, \lambda_{2}...\lambda_{m_{\psi}}\}\subseteq\Lambda$ <br>
$\Psi^{*}$, the set of load source *groups* where each load source *group* contains one and only one load source ($\Psi^{*}\subset\Psi\quad\mid\quad\left\vert\psi^{*}\right\vert=1 \quad\forall \psi^{*}\in\Psi^{*} $) <br>
$\psi_{\lambda}^{*}$, the load source *group* containing only the load source $\lambda$ <br>

In [7]:
loadsrcs = ampl.getSet('LOADSRCS')
loadsrcs.setValues(TblLoadSource.loadsourceid.tolist())
#print(loadsrcs.getValues())

loadsrcgrps = ampl.getSet('LOADSRCGRPS')
loadsrcgrps.setValues(TblLoadSourceGroup.loadsourcegroupid.tolist())
#print(loadsrcgrps.getValues())

loadsrcgrping = ampl.getSet('LOADSRCGRPING')
loadsrcgrping.setValues(list(zip(TblLoadSourceGroupLoadSource.loadsourcegroupid.tolist(),
                             TblLoadSourceGroupLoadSource.loadsourceid.tolist())))
#print(loadsrcgrping.getValues())

**BMPs and their groups**

$B$, a set of BMPs $b$ <br>
$\Gamma$, a set of BMP *groups* $\gamma$, where $\gamma=\{b_{1}, b_{2}...b_{n_{\gamma}}\}\subseteq B$<br>
$\gamma_{b}$, the BMP *group* to which BMP $b$ belongs

In [8]:
bmps = ampl.getSet('BMPS')
bmps.setValues(TblBmp.bmpid.tolist())
# print(bmps.getValues())

bmpgrps = ampl.getSet('BMPGRPS')
bmpgrps.setValues(TblBmpGroup.bmpgroupid.tolist())
# print(bmpgrps.getValues())

bmpgrping = ampl.getSet('BMPGRPING')
bmpgrping.setValues(list(zip(TblBmp.bmpid.tolist(),
                             TblBmp.bmpgroupid.tolist())))
# print(bmpgrping.getValues())

**PARAMETERS**

$y$, the scenario base year (defines the available load source acres ($T$) and their base loads ($\phi$))<br>
$cp$, the cost profile (defines the costs ($c$))

$c_{b}$, the cost per acre of BMP $b$ <br>

In [9]:
costprofileid=4

c = ampl.getParameter('c')
df = TblCostBmpLand[TblCostBmpLand['costprofileid']==costprofileid]
c.setValues(dict(zip(df.bmpid, df.totalannualizedcostperunit)))
# print(c.getValues())

$E_{b,p,l,\lambda}$, the effectiveness per acre of BMP $b$ on reducing pollutant $p$, in land-river segment $l$ and load source $\lambda$ <br>

In [17]:
E = ampl.getParameter('E')

effsubtable = TblBmpEfficiency[TblBmpEfficiency['lrsegid'].isin(lrsegsetlist)]
# make the pollutant names into an index instead of separate columns
listofdataframes = []
pltntdict = {'tn': 'N', 'tp': 'P', 'sed': 'S'}
for ps in ['tn', 'tp', 'sed']:
    bmpeff = effsubtable.loc[:, ['bmpid', 'lrsegid', 'loadsourceid', ps]]
    bmpeff['pltnt'] = pltntdict[ps]
    bmpeff.rename(columns={ps: 'effvalue'}, inplace=True)
    listofdataframes.append(bmpeff)

df = pd.concat(listofdataframes)
# Convert groups to dictionary ( with tuple->effvalue structure ) 
grouped = df.groupby(['bmpid', 'pltnt', 'lrsegid', 'loadsourceid'])
Edict = grouped['effvalue'].apply(lambda x: list(x)[0]).to_dict()

# display 5 keys for illustration
for k, v in zip(nrandkeys, [Edict[x] for x in list(islice(Edict,5))]):
    print(k, v)
    
E.setValues(Edict)

(3, 'N', 1677, 1) 0.03
(3, 'N', 1677, 2) 0.08
(3, 'N', 1677, 3) 0.08
(3, 'N', 1677, 4) 0.08
(3, 'N', 1677, 5) 0.03


$\tau_{l,p}$, the target percent load reduction per pollutant per land river segment

In [21]:
tau = ampl.getParameter('tau')

Taudict = {}
for l in lrsegsetlist:
    Taudict[(l, 'N')] = 0.5
    Taudict[(l, 'P')] = 0.5
    Taudict[(l, 'S')] = 0.5
display(Taudict)

tau.setValues(Taudict)

{(1677, 'N'): 0.5, (1677, 'P'): 0.5, (1677, 'S'): 0.5}

$T$, 

In [ ]:
ampl.readData(f_dat)

**Solve the problem**

In [ ]:
ampl.solve()